In [22]:
%matplotlib widget
''' 系统库 '''
import datetime
import time
from random import randint 
from importlib import reload 
# from numba import jit
# @jit(nopython=True)

import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt

''' 业务库 '''
# import mplfinance as mpf
import tushare as ts
ts.set_token('11137efdbeac800606d677871a55b3fd5aef79890c59867a8f34d03e')
pro = ts.pro_api()

''' 自建库 '''
from qflib import basic,syncdata
reload(basic)
reload(syncdata)

global engine, conn 
engine = basic.engine()
conn = basic.conn(engine)

trade_date = datetime.datetime.now().strftime('%Y%m%d')
print('date : ', trade_date)
run_static, run_dynamic = False, True
# run_static, run_dynamic = True, True
# run_static, run_dynamic = True, False


date :  20230718


In [23]:
# 分钟数据，freq/数据频度 ：1MIN表示1分钟（1/5/15/30/60分钟） D日线 ，默认D， W，M;  *** 您每天最多访问该接口2次
# df = ts.pro_bar(ts_code='601318.SH', adj='qfq', freq='1min', start_date='20220401', end_date='20220430')
# print(df.head(3))
# df.to_csv('../data/minite_601318.csv')

# print( '更新基本信息' )
# df = pro.stock_basic(exchange='', list_status='L')
# syncdata.sync_basic(engine, df, 'stock_basic', trade_date)

print( "更新深股通清单")
df = pro.hs_const(hs_type='SZ')
syncdata.sync_basic(engine, df, 'sl_shengutong', trade_date)

# df = ts.pro_bar(ts_code='601318.SH', adj='qfq', start_date='20200101', end_date='20231231')
# df
# basic.write_data(engine,df,'sl_shengutong', False)

更新深股通清单
sl_shengutong <sqlalchemy.engine.cursor.LegacyCursorResult object at 0x000001DF62EECA00>


In [24]:
%%time
"""
静态数据更新
"""

if run_static:

    print( '更新基本信息' )
    df = pro.stock_basic(exchange='', list_status='L')
    syncdata.sync_basic(engine, df, 'stock_basic', trade_date)

    print( "更新公司信息")
    df = pro.stock_company(exchange='', ts_code='')  
    syncdata.sync_basic(engine, df, 'stock_company', trade_date)
    
    print( "获得股票关键市场信息； 5000积分以上，否则每分钟最多2次, 每天20次；")
    df = pro.bak_basic(trade_date=trade_date)
    syncdata.sync_basic(engine, df, 'bak_basic', trade_date)

    print( "更新深股通清单")
    df = pro.hs_const(hs_type='SZ')
    syncdata.sync_basic(engine, df, 'sl_shengutong', trade_date)

    print("更新沪股通清单")
    df = pro.hs_const(hs_type='SH')
    syncdata.sync_basic(engine, df, 'sl_hugutong', trade_date)

    print("开始改名的股票")
    df = pro.namechange(ts_code='', start_date='2019-01-01', fields='ts_code,name,start_date,end_date,change_reason')
    syncdata.sync_basic(engine, df, 'namechange', trade_date)

    print("交易日历")
    df = pro.trade_cal(exchange='SSE', start_date='20190101', end_date='20221231')
    syncdata.sync_basic(engine, df, 'calendar', trade_date)
    
    print("股票概念清单")   # 879+个
    df = pro.concept(src='ts') 
    syncdata.sync_basic(engine, df, 'concept', trade_date)

    

CPU times: total: 0 ns
Wall time: 0 ns


In [26]:
%%time
"""
动态数据更新：每日成交、每日MoneyFlow
"""

# days = 365*5
days = int(365 * 7)

reload(basic)
reload(syncdata)

if run_dynamic:
    # 每日成交数据 -- OK
    syncdata.sync_tran_daily(conn, days)
    # 每日资金流向  --  暂停
    # syncdata.sync_moneyflow(engine, days)

    # 1、国内股市：可显示复权后和复权前信息, 现在用前复权价格
    # df = ts.pro_bar(ts_code=sl_list['ts_code'][i], asset='E', adj='qfq', start_date=start_date, end_date='')

20160720  - read:  (2619, 11)  - Added 2619
20160721  - read:  (2620, 11)  - Added 2620
20160722  - read:  (2622, 11)  - Added 2622
20160723  - read:  (0, 11)  - none 
20160724  - read:  (0, 11)  - none 
20160725  - read:  (2622, 11)  - Added 2622
20160726  - read:  (2632, 11)  - Added 2632
20160727  - read:  (2637, 11)  - Added 2637
20160728  - read:  (2643, 11)  - Added 2643
20160729  - read:  (2644, 11)  - Added 2644
20160730  - read:  (0, 11)  - none 
20160731  - read:  (0, 11)  - none 
20160801  - read:  (2647, 11)  - Added 2647
20160802  - read:  (2656, 11)  - Added 2656
20160803  - read:  (2653, 11)  - Added 2653
20160804  - read:  (2653, 11)  - Added 2653
20160805  - read:  (2656, 11)  - Added 2656
20160806  - read:  (0, 11)  - none 
20160807  - read:  (0, 11)  - none 
20160808  - read:  (2662, 11)  - Added 2662
20160809  - read:  (2668, 11)  - Added 2668
20160810  - read:  (2672, 11)  - Added 2672
20160811  - read:  (2673, 11)  - Added 2673
20160812  - read:  (2680, 11)  - Add

In [ ]:

# import cStringIO
 
# output = cStringIO.StringIO()
# # ignore the index
# df_a.to_csv(output, sep='\t',index = False, header = False)
# output.getvalue()
# # jump to start of stream
# output.seek(0)
 
# connection = engine.raw_connection() #engine 是 from sqlalchemy import create_engine
# cursor = connection.cursor()
# # null value become ''
# cursor.copy_from(output,table_name,null='')
# connection.commit()


In [ ]:
'''管理层薪酬和持股; 需要高级权限 '''
# df = pro.stk_rewards(ts_code='601318.SH')

'''--- 新股上市清单， 120积分 '''
# df = pro.new_share(start_date='20220101', end_date='20221231')
# df.tail(10)

''' 概念股清单 
    58,TS58,锂电隔膜,ts
    59,TS59,锂电设备,ts
    60,TS60,锂电原料,ts
    61,TS61,锂电制造,ts
'''
# df = pro.concept_detail(id='TS60', fields='ts_code,name') 
# df.to_csv('~/win-desk/lidianyuanliao.txt', index=False, header=False)
